## Predicting sale price
### This notebook contains the necessary code for predicting sale price with ML

Importing packages

In [28]:
try:
    import pandas as pd
    import plotly.express as px 
    import numpy as np
    import tensorflow as tf
    from sklearn.model_selection import train_test_split
    import sklearn
except:
    !pip install scikit-learn
    !pip install pandas
    !pip install plotly
    !pip install tensorflow
    import pandas as pd
    import plotly.express as px 
    import numpy as np
    import tensorflow as tf
    from sklearn.model_selection import train_test_split
    import sklearn

Defining the modell

In [29]:
def createModel(normlayer,trainshape1):
    model=tf.keras.Sequential([
        normlayer,
        tf.keras.layers.Dense(256,activation='relu',input_shape=(trainshape1,)),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(1)
        
        
    ])
    model.compile(optimizer='adam',loss='mse',metrics=['mae'])
    return model

Loading data and removing KitchenQual since I have concluded that this knowledge doesn't improve the results as significant as it makes the program more complex 

In [30]:
data = pd.read_pickle('../data/clean_housing_data.pkl')
a=data.pop('KitchenQual')

In [31]:
data.columns

Index(['1stFlrSF', 'GarageArea', 'GarageYrBlt', 'GrLivArea', 'LotArea',
       'MasVnrArea', 'OpenPorchSF', 'OverallQual', 'TotalBsmtSF', 'YearBuilt',
       'YearRemodAdd', 'SalePrice'],
      dtype='object')

Splitting the data to feature and target sets

In [32]:
features = data.copy()
target=features.pop('SalePrice')

Save the list of important features

In [33]:
import pickle
with open('../data/important_features.pkl','wb') as f:
    pickle.dump(list(features.columns),f)

Creating test and train sets

In [34]:
X_train,X_test,Y_train,Y_test=train_test_split(features,target,test_size=.1)

Setting up the normalisation layer

In [35]:
normlayer=tf.keras.layers.Normalization()
normlayer.adapt(X_train)

Create and fitting the modell

In [36]:
es=tf.keras.callbacks.EarlyStopping(patience=10)
model=createModel(normlayer,X_train.shape[1])
model.fit(X_train,Y_train,batch_size=4,epochs=50,verbose=1,validation_split=.2,callbacks=[es])

Epoch 1/50
205/205 [==============================] - 7s 16ms/step - loss: 28655360000.0000 - mae: 157214.1562 - val_loss: 7091656192.0000 - val_mae: 75961.6328
Epoch 2/50
205/205 [==============================] - 3s 13ms/step - loss: 3266996992.0000 - mae: 46179.3125 - val_loss: 2256882688.0000 - val_mae: 37347.0625
Epoch 3/50
205/205 [==============================] - 1s 6ms/step - loss: 1756648832.0000 - mae: 32915.0469 - val_loss: 1436691712.0000 - val_mae: 28929.8828
Epoch 4/50
205/205 [==============================] - 1s 6ms/step - loss: 1181229568.0000 - mae: 26743.4375 - val_loss: 1072522880.0000 - val_mae: 25145.9238
Epoch 5/50
205/205 [==============================] - 1s 6ms/step - loss: 930410240.0000 - mae: 23613.4746 - val_loss: 931235648.0000 - val_mae: 22401.5898
Epoch 6/50
205/205 [==============================] - 1s 6ms/step - loss: 826230912.0000 - mae: 22211.7012 - val_loss: 920104576.0000 - val_mae: 22774.4609
Epoch 7/50
205/205 [==============================] 

### Evaluating the modell

Evaluate on test set given the metrics Mean Squared Error and Mean Absolute Error

In [37]:
model.evaluate(X_test,Y_test)

4/4 [==============================] - 1s 3ms/step - loss: 510480192.0000 - mae: 17000.2891


[510480192.0, 17000.2890625]

Calculating R2

In [38]:
Y_test_prediction=model.predict(X_test).flatten()
Y_train_prediction= model.predict(X_train).flatten()

import pickle

with open('../data/Y_test_prediction.pkl','wb') as f:
    pickle.dump(list(Y_test_prediction),f)

with open('../data/Y_train_prediction.pkl','wb') as f:
    pickle.dump(list(Y_train_prediction),f)
    
with open('../data/Y_test.pkl','wb') as f:
    pickle.dump(list(Y_test),f)
    
with open('../data/Y_train.pkl','wb') as f:
    pickle.dump(list(Y_train),f)

r2_scores=[sklearn.metrics.r2_score(list(Y_test_prediction),list(Y_test)),
           sklearn.metrics.r2_score(list(Y_train_prediction),list(Y_train))]

print(f'R2 on test set: {r2_scores[0]}')
print(f'R2 on train set: {r2_scores[1]}')

with open('../data/r2_scores.pkl','wb') as f:
    pickle.dump(r2_scores,f)

32/32 [==============================] - 2s 1ms/step
R2 on test set: 0.8131592506600557
R2 on train set: 0.8164066285069682


Plotting the actual and the predicted values

In [39]:
fig=px.scatter({'Actual':list(Y_test),'Predicted':list(Y_test_prediction)},
           x='Actual',
           y='Predicted',
           range_x=[50_000,400_000],range_y=[50_000,400_000])

fig.add_shape(type='line',
                x0=70_000,
                y0=70_000,
                x1=380_000,
                y1=380_000,
                line=dict(color='Red',),
                xref='x',
                yref='y'
)
fig